In [9]:
import numpy as np
import pandas as pd
import yfinance as yf
import datetime
import copy
import matplotlib.pyplot as plt

In [7]:
import datetime as dt

In [15]:
tickers = ["AMZN","MSFT","INTC","GOOG"]
start = datetime.datetime.today() - dt.timedelta(5000)
end = datetime.date.today()
cl_price = pd.DataFrame()#empty dataframe to be filled with data prices
ohlcv_data = {} # empty dictionary
# looping over tickers and creating dataframe with close prices
for ticker in tickers:
    ohlcv_data[ticker] = yf.download(ticker,start,end,interval='1d')
    ohlcv_data[ticker].dropna(inplace=True,how="all")
 
tickers = ohlcv_data.keys() # redefine tickers variable after removing any tickers with corrupted data

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [16]:
google = ohlcv_data['GOOG']

In [17]:
microsoft = ohlcv_data['MSFT']

In [18]:
intel = ohlcv_data['INTC']

In [19]:
len(intel)

3445

In [20]:
amazon = ohlcv_data['AMZN']

In [21]:
def volatility(DF):
    "function to calculate annualized volatility of a trading strategy"
    df = DF.copy()
    df["daily_ret"] = DF["Adj Close"].pct_change()*100
    vol = df["daily_ret"].std()
    df.dropna(inplace=True)
    return vol

In [24]:
for ticker in ohlcv_data:
    ohlcv_data[ticker]['vol']= volatility(ohlcv_data[ticker],)

In [25]:
microsoft.head()

,Open,High,Low,Close,Adj Close,Volume,vol
Date,,,,,,,
2009-05-20,20.410000,20.690001,20.219999,20.379999,15.439422,47450700.0,1.635987
2009-05-21,20.139999,20.230000,19.510000,19.820000,15.015175,58232800.0,1.635987
2009-05-22,19.930000,20.170000,19.469999,19.750000,14.962147,37318700.0,1.635987
2009-05-26,19.540001,20.450001,19.450001,20.340000,15.409123,44991700.0,1.635987
2009-05-27,20.250000,20.600000,20.070000,20.129999,15.250027,42892900.0,1.635987


In [26]:
def BollBnd(DF,n = 14):
    df = DF.copy()
    df["MA"]=df["Adj Close"].rolling(n).mean()
    df["BB_up"] = df["MA"] + 2*df['MA'].rolling(n).std()
    df["BB_dn"] = df["MA"] - 2*df['MA'].rolling(n).std()
    df["BB_width"] = df["BB_up"] - df["BB_dn"]
    df.dropna(inplace=True)
    return df[["MA", "BB_up", "BB_dn","BB_width"]]


In [27]:
for ticker in ohlcv_data:
    ohlcv_data[ticker][["MA", "BB_up", "BB_dn","BB_width"]]= BollBnd(ohlcv_data[ticker],)

In [28]:
def OBV(DF):
    """function to calculate On Balance Volume"""
    df = DF.copy()
    df['daily_ret'] = df['Adj Close'].pct_change()
    df['direction'] = np.where(df['daily_ret']>=0,1,-1)
    df['direction'][0] = 0
    df['vol_adj'] = df['Volume'] * df['direction']
    df['obv'] = df['vol_adj'].cumsum()
    df.dropna(inplace=True)
    return df['obv']

In [29]:
for ticker in ohlcv_data:
    ohlcv_data[ticker]['obv']= OBV(ohlcv_data[ticker],)

C:\Users\sinti\AppData\Local\Temp/ipykernel_12152/2620105290.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['direction'][0] = 0


In [67]:
microsoft.describe()  

,Open,High,Low,Close,Adj Close,Volume,MA,BB_up,BB_dn,BB_width,obv,vol
count,1389.000000,1389.000000,1389.000000,1389.000000,1389.000000,1.389000e+03,1363.000000,1363.000000,1363.000000,1363.000000,1.389000e+03,1.389000e+03
mean,184.459100,186.371389,182.432232,184.491886,180.691427,2.945575e+07,182.042183,186.640392,177.443973,9.196419,1.653764e+09,1.895947e+00
std,78.124926,78.975841,77.203142,78.129488,79.081260,1.278681e+07,78.427055,80.843680,76.146080,8.039909,9.011970e+08,6.819225e-14
min,71.610001,72.190002,71.279999,71.410004,66.557220,7.425600e+06,67.865311,68.255160,67.436355,0.212081,-1.906304e+08,1.895947e+00
25%,108.559998,109.580002,107.510002,108.489998,103.576118,2.144690e+07,103.845098,105.483937,102.469380,3.348886,7.263833e+08,1.895947e+00
50%,180.000000,181.990005,176.539993,179.899994,175.109482,2.632920e+07,175.758391,181.350196,167.630369,6.499013,1.832233e+09,1.895947e+00
75%,249.979996,252.080002,247.169998,250.660004,248.225189,3.362180e+07,249.210711,256.376819,242.852970,12.769424,2.520556e+09,1.895947e+00
max,344.619995,349.670013,342.200012,343.109985,339.924866,1.112421e+08,333.910322,347.320221,329.199517,38.226743,2.891288e+09,1.895947e+00


In [30]:
import scipy
import sklearn
import statsmodels.api as sm
import numpy as np
import matplotlib as plt
import seaborn as sns

In [31]:
y = microsoft['Adj Close']
x1 = microsoft[['High','Low','Volume']]
x = sm.add_constant(x1)
results = sm.OLS(y,x).fit()

In [32]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Adj Close   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 5.221e+06
Date:                Thu, 26 Jan 2023   Prob (F-statistic):               0.00
Time:                        20:44:29   Log-Likelihood:                -5840.4
No. Observations:                3445   AIC:                         1.169e+04
Df Residuals:                    3441   BIC:                         1.171e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -5.7284      0.065    -88.552      0.000      -5.855      -5.602
High           0.5956      0.016     38.141      0.000       0.565       0.626
Low            0.4144      0.016     25.893      0.000       0.383       0.446
Volume     -5.177e-09   1.16e-09     -4.478      0.000   -7.44e-09   -2.91e-09
==============================================================================
Omnibus:                      622.555   Durbin-Watson:                   1.520
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            11077.234
Skew:                           0.324   Prob(JB):                         0.00
Kurtosis:                      11.761   Cond. No.                     1.30e+08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.3e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [33]:
ohlcv_data['MSFT']

,Open,High,Low,Close,Adj Close,Volume,vol,MA,BB_up,BB_dn,BB_width,obv
Date,,,,,,,,,,,,
2009-05-20,20.410000,20.690001,20.219999,20.379999,15.439422,47450700.0,1.635987,NaN,NaN,NaN,NaN,NaN
2009-05-21,20.139999,20.230000,19.510000,19.820000,15.015175,58232800.0,1.635987,NaN,NaN,NaN,NaN,NaN
2009-05-22,19.930000,20.170000,19.469999,19.750000,14.962147,37318700.0,1.635987,NaN,NaN,NaN,NaN,NaN
2009-05-26,19.540001,20.450001,19.450001,20.340000,15.409123,44991700.0,1.635987,NaN,NaN,NaN,NaN,NaN
2009-05-27,20.250000,20.600000,20.070000,20.129999,15.250027,42892900.0,1.635987,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-19,233.779999,235.520004,230.679993,231.929993,231.929993,28623000.0,1.635987,233.879999,242.054642,225.705356,16.349286,3.941021e+09
2023-01-20,234.860001,240.740005,234.509995,240.220001,240.220001,35323400.0,1.635987,233.823571,241.095803,226.551340,14.544463,3.976345e+09
2023-01-23,241.100006,245.169998,239.649994,242.580002,242.580002,31934000.0,1.635987,234.020714,240.006532,228.034896,11.971636,4.008279e+09
